In [106]:
import numpy as np
import scipy as sp
import pandas as pd
from matplotlib import pyplot as plt
import math

import random
import itertools
import timeit

import networkx as nx
import netlsd
import distanceclosure as dc
from portrait_divergence import portrait_divergence

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [3]:
def weighten(G):
    for (u, v, w) in G.edges(data=True):
        w['weight'] = np.random.uniform()
    return G

In [4]:
V = 1000
D = 0.01
E = round(D*V*(V-1)/2)

G = nx.barabasi_albert_graph(V, round(E/V))
G = weighten(G)

In [129]:
if 3 < 5 < 8:
    print("yay")

yay


In [252]:
for i in range(100):
    while True:
        try:
            tau1 = 3
            tau2 = 1.1
            mu = 0.2
            G = nx.LFR_benchmark_graph(
                n, tau1, tau2, mu, average_degree=10, max_degree=100, min_community = 5)
            break
        except:
            continue
    print(G)

Graph with 1000 nodes and 5602 edges
Graph with 1000 nodes and 5638 edges
Graph with 1000 nodes and 5490 edges
Graph with 1000 nodes and 5579 edges
Graph with 1000 nodes and 5493 edges
Graph with 1000 nodes and 5786 edges
Graph with 1000 nodes and 5452 edges
Graph with 1000 nodes and 5389 edges
Graph with 1000 nodes and 5693 edges
Graph with 1000 nodes and 5342 edges
Graph with 1000 nodes and 5436 edges
Graph with 1000 nodes and 5303 edges
Graph with 1000 nodes and 5361 edges
Graph with 1000 nodes and 5515 edges
Graph with 1000 nodes and 5630 edges
Graph with 1000 nodes and 5418 edges
Graph with 1000 nodes and 5326 edges
Graph with 1000 nodes and 5779 edges
Graph with 1000 nodes and 5450 edges
Graph with 1000 nodes and 5847 edges
Graph with 1000 nodes and 5527 edges
Graph with 1000 nodes and 5514 edges
Graph with 1000 nodes and 5753 edges
Graph with 1000 nodes and 5408 edges
Graph with 1000 nodes and 5399 edges
Graph with 1000 nodes and 5615 edges
Graph with 1000 nodes and 5588 edges
G

In [5]:
def removal(H):
    H.remove_edge(*random.choice(list(H.edges)))

def addition(H):
    u, v = random.choice(list(H.nodes)), random.choice(list(H.nodes))
    while (u, v) in H.edges:
        u, v = random.choice(list(H.nodes)), random.choice(list(H.nodes))
    H.add_edge(u, v, weight = np.random.uniform())
    
def random_switching(H):
    removal(H)
    addition(H)

def degree_preserving_switching(H):
    a, b = random.choice(list(H.edges))
    c, d = random.choice(list(H.edges))
    while (a, c) in H.edges or (b, d) in H.edges:
        a, b = random.choice(list(H.edges))
        c, d = random.choice(list(H.edges))

    H.remove_edge(a, b)
    H.remove_edge(c, d)
    H.add_edge(a, c, weight = np.random.uniform())
    H.add_edge(b, d, weight = np.random.uniform())

In [14]:
def euc_distance(m, H, G, mG = None):
    if mG is None:
        mG = m(G)
    return np.linalg.norm(m(H) - mG)

def lap_spec_d(H, G, mG = None):
    return euc_distance(nx.laplacian_spectrum, H, G, mG)

def adj_spec_d(H, G, mG = None):
    return euc_distance(nx.adjacency_spectrum, H, G, mG)

def nlap_spec_d(H, G, mG = None):
    return euc_distance(nx.normalized_laplacian_spectrum, H, G, mG)

def netlsd_heat_d(H, G, mG = None):
    return euc_distance(netlsd.heat, H, G, mG)

def portrait_div_d(H, G):
    return portrait_divergence(H, G)

In [7]:
def test(G, perturbation, metrics, K, N):
    start = timeit.default_timer()
    apsp_G = dc.metric_backbone(G, weight='weight')
    
    Distances_full = {m_id : [ [] for _ in range(K) ] for m_id, _, _ in metrics}
    Distances_apsp = {m_id : [ [] for _ in range(K) ] for m_id, _, _ in metrics}
    
    prec_full = {}
    prec_apsp = {}
    
    for m_id, md, m in metrics:
        if m is not None:
            prec_full[m_id] = m(G)
            prec_apsp[m_id] = m(apsp_G)
            md_f = md(G, G, prec_full[m_id])
            md_a = md(apsp_G, apsp_G, prec_apsp[m_id])
            for i in range(K):
                Distances_full[m_id][i].append(md_f)
                Distances_apsp[m_id][i].append(md_a)
        else:
            md_f = md(G, G)
            md_a = md(apsp_G, apsp_G)
            for i in range(K):
                Distances_full[m_id][i].append(md_f)
                Distances_apsp[m_id][i].append(md_a)
    for i in range(K):
        print(f'K = {i}')
        H = G.copy()
        for j in range(N):
            print(f'{j}\'th perturbation')
            perturbation(H)
            print(f'Metric backbone start: {timeit.default_timer() - start:.2f} s')
            apsp_H = dc.metric_backbone(H, weight='weight')
            print(f'Metric backbone finish: {timeit.default_timer() - start:.2f} s')
            
            for m_id, md, m in metrics:
                if m is not None:
                    Distances_full[m_id][i].append(md(H, G, prec_full[m_id]))
                    Distances_apsp[m_id][i].append(md(apsp_H, apsp_G, prec_apsp[m_id]))
                else:
                    print(f'Portrait start: {timeit.default_timer() - start:.2f} s')
                    Distances_full[m_id][i].append(md(H, G))
                    Distances_apsp[m_id][i].append(md(apsp_H, apsp_G))
                    print(f'Portrait finish: {timeit.default_timer() - start:.2f} s')
    
    return Distances_full, Distances_apsp

In [8]:
metrics = [
    ("lap", lap_spec_d, nx.laplacian_spectrum), 
    ("adj", adj_spec_d, nx.adjacency_spectrum),
    ("nlap", nlap_spec_d, nx.normalized_laplacian_spectrum),
    ("netlsd", netlsd_heat_d, netlsd.heat),
    ("portrait", portrait_div_d, None)
]

In [18]:
N = 4
K = 1

Distances_full, Distances_apsp = test(G, removal, metrics, K, N)

K = 0
0'th perturbation
Metric backbone start: 25.30 s
Metric backbone finish: 32.17 s
Portrait start: 35.09 s
Portrait finish: 44.61 s
1'th perturbation
Metric backbone start: 44.61 s
Metric backbone finish: 51.77 s
Portrait start: 55.48 s
Portrait finish: 67.98 s
2'th perturbation
Metric backbone start: 67.98 s
Metric backbone finish: 75.13 s
Portrait start: 80.59 s
Portrait finish: 92.19 s
3'th perturbation
Metric backbone start: 92.20 s
Metric backbone finish: 100.46 s
Portrait start: 103.37 s
Portrait finish: 113.72 s


In [23]:
H = G.copy()
removal(H)
print(G)
print(H)
print(np.linalg.norm(nx.adjacency_spectrum(G) - nx.adjacency_spectrum(H)))
print(np.linalg.norm(nx.laplacian_spectrum(G) - nx.laplacian_spectrum(H)))

Graph with 1000 nodes and 4975 edges
Graph with 1000 nodes and 4974 edges
76.53999613926835
0.6884989048977289


In [10]:
df = pd.concat({k : pd.DataFrame(a).T.agg(['mean', 'std'], axis=1) for k, a in Distances_full.items()}, axis=1)
df

adj    
    mean std
0  0.000 nan
1 73.852 nan
2 74.061 nan
3 73.268 nan
4 78.102 nan

In [11]:
#df.to_csv("results.csv")